<a href="https://colab.research.google.com/github/hiroalchem/data_science_lecture/blob/main/evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# データサイエンス講義第三回
これはデータサイエンス講義第三回用スクリプトです   
Colaboratoryのハードウェア アクセラレータ設定は今回はCPU (None) にして実行してください   
上のランタイムタブからランタイムのタイプを変更を選択してください   
Google Drive をマウントしてください

このスクリプトでは学習結果の評価を行ってみます

In [1]:
import os
import pathlib

In [2]:
# 必要なライブラリをインストールします
!pip install pycocotools
!pip install tf_slim
!pip install lvis
!pip install tensorflow==2.2

     |████████████████████████████████| 358kB 5.2MB/s 
     |████████████████████████████████| 516.2MB 32kB/s 
     |████████████████████████████████| 3.0MB 45.7MB/s 
     |████████████████████████████████| 460kB 43.4MB/s 
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0


In [3]:
!git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 2322, done.
remote: Counting objects: 100% (2322/2322), done.
remote: Compressing objects: 100% (2026/2026), done.
remote: Total 2322 (delta 562), reused 923 (delta 271), pack-reused 0
Receiving objects: 100% (2322/2322), 30.65 MiB | 28.48 MiB/s, done.
Resolving deltas: 100% (562/562), done.


In [4]:
# 必要なものをインストールします
# 少し時間がかかります
%%bash
cd /content/models/research
protoc object_detection/protos/*.proto --python_out=.
pip install .

Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-cp36-none-any.whl size=1376510 sha256=2c5783d4bb556002fa29af74be01dfb20aefed0c8acce686299464501f7e1261
  Stored in directory: /tmp/pip-ephem-wheel-cache-w4iil1wi/wheels/94/49/4b/39b051683087a22ef7e80ec52152a27249d1a644ccf4e442ea
Successfully built object-detection


In [ ]:
# install できたか確認
%cd /content/models/research
%set_env PYTHONPATH=/content/models/research:/content/models/research/slim
os.environ['PYTHONPATH'] += ":/content/models" 
import sys
sys.path.append("/content/models")
!python ./object_detection/builders/model_builder_tf2_test.py

/content/models/research
env: PYTHONPATH=/content/models/research:/content/models/research/slim
Running tests under Python 3.6.9: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_model
2020-12-08 08:45:48.003583: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-12-08 08:45:48.077931: E tensorflow/stream_executor/cuda/cuda_driver.cc:313] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2020-12-08 08:45:48.077994: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (9c66354e2de3): /proc/driver/nvidia/version does not exist
2020-12-08 08:45:48.078372: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2020-12-08 08:45:48.084389: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 230000000

In [ ]:
%cd /content/models/research
!python object_detection/model_main_tf2.py \
    --pipeline_config_path="/content/drive/MyDrive/lecture2020/trial1/mypipeline.config" \
    --model_dir="/content/drive/MyDrive/lecture2020/trial1/models/faster_rcnn_resnet50_v1_800x1333_coco17_gpu-8" \
    --checkpoint_dir="/content/drive/MyDrive/lecture2020/trial1/models/faster_rcnn_resnet50_v1_800x1333_coco17_gpu-8" \
    --alsologtostderr

Avereage PrecisionやAverage Recallが出てきたと思います   
出てきたら上のセルは止めてください

この中で一番よく使われるのは Average Precision (AP)です   
APとは簡単には推論したboxがどのくらい正解だったかの指標です   
詳細はこちらなどを読んで見てください
https://qiita.com/mdo4nt6n/items/08e11426e2fac8433fed  
これを求めるには推論したboxがどの程度正解のboxと重なっていた場合に正解とするかの閾値を決める必要があります   
それが IoU (Intersection over Union) 
一番わかりやすいのは2つ目のIoU=0.50と書いてある項目で、つまり半分以上重なっていたら正解とする時のAPを求めた時の数値、のようなものです


学習step数を増やした場合や、別のデータセットで見た目の結果とAPの数値を比較してみてください